In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
jan_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet')
feb_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-02.parquet')

In [3]:
print("Number of columns in January DataFrame:", len(jan_df.columns))

Number of columns in January DataFrame: 20


In [4]:
jan_df['duration'] = (jan_df['tpep_dropoff_datetime'] - jan_df['tpep_pickup_datetime']).dt.total_seconds() / 60
feb_df['duration'] = (feb_df['tpep_dropoff_datetime'] - feb_df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [5]:
print("Standard deviation of duration in January DataFrame:", jan_df['duration'].std())

Standard deviation of duration in January DataFrame: 38.71358219498144
